In [4]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('/home/aaron/Project/MyOwnStuff/STAT/individual_data_train.csv')
Class_df = np.array(train_df['Class'])
train_df = train_df.drop(['Class'], 1)
Fnlwgt_df = train_df['Fnlwgt']
Fnlwgt_df = np.array(Fnlwgt_df / Fnlwgt_df.max())
test_df = pd.read_csv('/home/aaron/Project/MyOwnStuff/STAT/individual_data_test.csv')
split_col = train_df.shape[0]
grand_df = pd.get_dummies(pd.concat([train_df, test_df]))
train_df = np.array(grand_df[:split_col])
test_df = np.array(grand_df[split_col:])
sample_df = pd.read_csv('/home/aaron/Project/MyOwnStuff/STAT/individual_data_sample_submission.csv')
sample_df.head()

print(train_df.shape,Class_df.shape)

def Sigmoid(w,x):
    return 1 / (1 + np.exp(-w.dot(x)))

class LR():
    def train(self, data, label, wgt, num_it, _lambda=1):
        data=np.insert(data,0,1,axis=1)
        self.Weight = np.zeros(data.shape[1])
        self.R=np.zeros([data.shape[0],data.shape[0]])
        for i in range(num_it):
            print("{} iteration".format(i))
            L_1=0
            for j in range(data.shape[0]):
                L_1+=(Sigmoid(self.Weight,data[j])-label[j])*data[j]
                self.R[j][j]=Sigmoid(self.Weight,data[j])*(1-Sigmoid(self.Weight,data[j]))
            H=np.matmul(np.matmul(data.transpose(),self.R),data)+_lambda*np.identity(data.shape[1])
            self.Weight-=np.matmul(np.linalg.pinv(H),L_1+_lambda*self.Weight)
    def predict(self,x):
        return (Sigmoid(self.Weight,x) > 0.5)
    def test(self,data,label):
        data=np.insert(data,0,1,axis=1)
        return np.sum(np.equal(np.apply_along_axis(self.predict,1,data),label))/data.shape[0]
    def write(self,test):
        test=np.insert(test,0,1,axis=1)
        result = np.empty([test.shape[0],2],dtype=np.int32)
        for i in range(result.shape[0]):
            result[i][0]=i+1
            result[i][1]=self.predict(test[i])
        output=pd.DataFrame(data=result,columns=['id','Class'],dtype=np.int32)
        output.to_csv("result.csv",index=False)

L=LR()
L2=LR()
L.train(train_df, Class_df, Fnlwgt_df,10)
L2.train(train_df,Class_df,Fnlwgt_df,20)
print((L.Weight-L2.Weight)/L2.Weight)
print(L.test(train_df, Class_df))
print(L2.test(train_df, Class_df))
L2.write(test_df)

(32561, 108) (32561,)
0 iteration
1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
0 iteration
1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
11 iteration
12 iteration
13 iteration
14 iteration
15 iteration
16 iteration
17 iteration
18 iteration
19 iteration
[ 1.73733156e-12  5.99084008e-15 -7.18727109e-13  8.56606848e-13
  1.55436653e-14 -1.48063916e-14 -2.14617285e-14 -1.47283443e-12
  2.93797843e-12 -2.00981152e-12 -1.19560409e-14 -3.46180144e-12
 -1.47015840e-11 -1.07244901e-12 -1.52817058e-12 -1.32408211e-13
 -1.59761517e-12 -1.00063448e-12 -1.39476205e-12 -1.27975824e-11
 -5.90007953e-12 -2.33784463e-12 -1.98069266e-12  5.97912173e-13
  1.45245846e-12 -2.63282931e-12 -1.11287818e-12 -8.49531493e-13
 -1.72695439e-12  3.15957057e-13 -1.09384225e-12 -2.57659942e-13
 -7.02972235e-13  1.00440709e-12  9.30787501e-13 -7.43944372e-13
 -4.41854846e-13 -6.818